In [1]:
import girder_client
import json, os, re
import pandas as pd

tsApiKey='vsYwFv7qd7Lq3tLUp2YabwA50IEDaGshXqoQue34'
tsApiUrl='http://transplant.digitalslidearchive.emory.edu:8080/api/v1'

gc = girder_client.GirderClient(apiUrl=tsApiUrl)
gc.authenticate(apiKey=tsApiKey)

# accessionNumber  blockID  stainID
tpName_re  = re.compile('(?P<accessionNumber>S\d\d-\d+).(?P<blockID>.+)[_\s](?P<stainID>[a-zA-Z]+)\.svs')
transplantSlideCollection = '5cf820a91ee779b5180282c4'

md_df = pd.read_csv("./metada_EUH_complete.csv")
md_df.set_index('section_ID2',inplace=True)

In [14]:
manifestItems = []
noMatch = []
for f in gc.listFolder(transplantSlideCollection,parentFolderType='collection'):
    if f['name'].startswith('2020'):
        itemsInFolder = 0
        print(f['name'])
        childItems = gc.listResource("resource/%s/items?type=folder" % f['_id'])
        for i in childItems:
            meta = i['meta']
            if 'stain' in meta and meta['stain'] == 'PAS':
                manifestItems.append(i)
                fn = i['name']
                print(i['name'])
            else:
                if 'stainID' not in meta: ## means there was no metadata at all
                    n = i['name']
                    m = tpName_re.search(n)
                    if m:
                        print("\t",m.groupdict(),i['name'],meta)
                        gc.addMetadataToItem(i['_id'],dict(m.groupdict()))
                        
                        accessionNumber = m.groupdict()['accessionNumber']
                        if accessionNumber in md_df.index:
                            print(md_df.loc[accessionNumber])
                            print("ITS AN ADULT SLIDE")
                            sys.exit()

                    else:
                        noMatch.append(i['name'])

2020-12-07
	 {'accessionNumber': 'S13-1552', 'blockID': 'L5', 'stainID': 'HE'} S13-1552_L5 HE.svs {'accessionNumber': 'S13-1552'}
	 {'accessionNumber': 'S13-647', 'blockID': 'L9', 'stainID': 'HE'} S13-647_L9 HE.svs {'accessionNumber': 'S13-647'}
	 {'accessionNumber': 'S13-741', 'blockID': 'L1', 'stainID': 'HE'} S13-741_L1 HE.svs {'accessionNumber': 'S13-741'}
	 {'accessionNumber': 'S13-1552', 'blockID': 'L10', 'stainID': 'HE'} S13-1552_L10 HE.svs {'accessionNumber': 'S13-1552'}
	 {'accessionNumber': 'S13-1150', 'blockID': 'L5', 'stainID': 'HE'} S13-1150_L5 HE.svs {'accessionNumber': 'S13-1150'}
	 {'accessionNumber': 'S13-1489', 'blockID': 'L10', 'stainID': 'HE'} S13-1489_L10 HE.svs {'accessionNumber': 'S13-1489'}
	 {'accessionNumber': 'S13-1206', 'blockID': 'L10', 'stainID': 'HE'} S13-1206_L10 HE.svs {'accessionNumber': 'S13-1206'}
	 {'accessionNumber': 'S13-2573', 'blockID': 'L8', 'stainID': 'TRICH'} S13-2573_L8 TRICH.svs {'accessionNumber': 'S13-2573'}
	 {'accessionNumber': 'S13-670'

2020-12-12
2020-12-14
2020-12-15
2020-12-16
2020-12-18
2020-12-19
2020-12-21
2020-12-22
2020-12-23
2020-12-24


In [12]:
dict(m.groupdict())

{'accessionNumber': 'S13-1552', 'blockID': 'L5', 'stainID': 'HE'}